In [1]:
import glob
import pymysql
import datetime
import pandas as pd

from pathlib import Path

In [2]:
connection = pymysql.connect(host='dev-brandicommerce.ctfkpwjekkqd.ap-northeast-1.rds.amazonaws.com', port=3306, user='choisj2', passwd='brandi1234!', db='brandicommerce', charset='utf8')

In [3]:
def get_query(_from, _to):
    return """
SELECT 
    ORDER_REGIST_DATE,
    C_ORDRR_MBER_NO AS ORDRR_MBER_NO,
    COUNT(C_ORDER_NO) ORDER_CNT,
    SUM(C_ORDER_DETAIL_CNT) ORDER_DETAIL_CNT,
    SUM(C_ORDER_DETAIL_DSCNT_PC_SUM) ORDER_DETAIL_DSCNT_PC_SUM,
    SUM(C_ORDER_DETAIL_SELL_PC_SUM) ORDER_DETAIL_SELL_PC_SUM
FROM (
    SELECT
        C_ORDRR_MBER_NO,
        ORDER_REGIST_DATE,
        C_ORDER_NO,
        MAX(C_ORDER_DETAIL_CNT) AS C_ORDER_DETAIL_CNT,
        SUM(C_ORDER_DETAIL_DSCNT_PC) AS C_ORDER_DETAIL_DSCNT_PC_SUM,
        SUM(C_ORDER_DETAIL_SELL_PC) AS C_ORDER_DETAIL_SELL_PC_SUM
    FROM (
        SELECT 
            A.C_ORDER_NO,
            B.C_ORDRR_MBER_NO,
            B.C_ORDRR_TY_CD,       
            A.C_ORDER_DETAIL_CNT,
            D.C_ORDER_DETAIL_DSCNT_PC,
            D.C_ORDER_DETAIL_SELL_PC,
            DATE_FORMAT(A.C_ORDER_REGIST_DATETIME, '%Y-%m%-%d') AS ORDER_REGIST_DATE
        FROM      
            TB_C_ORDER AS A
            INNER JOIN TB_C_ORDERER B ON A.C_ORDRR_NO = B.C_ORDRR_NO
            INNER JOIN TB_C_ORDER_DETAIL C ON A.C_ORDER_NO = C.C_ORDER_NO
            INNER JOIN TB_C_ORDER_DETAIL_PRODUCT D ON C.C_ORDER_DETAIL_NO = D.C_ORDER_DETAIL_NO    
        WHERE
            A.C_ORDER_REGIST_DATETIME >= '{0}'  AND A.C_ORDER_REGIST_DATETIME <= '{1}'
    ) AS A
    GROUP BY ORDER_REGIST_DATE, C_ORDRR_MBER_NO, C_ORDER_NO
) AS A
GROUP BY C_ORDRR_MBER_NO, ORDER_REGIST_DATE
""".format(_from, _to)

In [4]:
# _timedelta = 100
_now = datetime.datetime.now()
# _from = (_now - datetime.timedelta(days=_timedelta)).strftime("%Y-%m-%d 00:00:00")
# _to = (_now - datetime.timedelta(days=_timedelta)).strftime("%Y-%m-%d 23:59:59")
#_from = datetime.datetime.now().strftime("%Y-%m-%d 00:00:00")
#_to = datetime.datetime.now().strftime("%Y-%m-%d 00:00:00")

In [5]:
def rfm(now):
    print(now, now[:4], now[5:7], now[8:10])
    df = pd.read_sql(get_query(now + ' 00:00:00', now + ' 23:59:59'), connection)
    df['ORDRR_MBER_NO'] = df['ORDRR_MBER_NO'].fillna(0)
    df['ORDRR_MBER_NO'] = df['ORDRR_MBER_NO'].astype('int32')
    df['ORDER_DETAIL_CNT'] = df['ORDER_DETAIL_CNT'].astype('int32')
    df['ORDER_DETAIL_DSCNT_PC_SUM'] = df['ORDER_DETAIL_DSCNT_PC_SUM'].astype('int32')
    df['ORDER_DETAIL_SELL_PC_SUM'] = df['ORDER_DETAIL_SELL_PC_SUM'].astype('int32')
    path = r'/Users/choisj2/data/ml-mart/rds/rfm/year={0}/month={1}/day={2}'.format(now[:4], now[5:7], now[8:10])
    Path(path).mkdir(parents=True, exist_ok=True)
    file_name = path + '/recency_frequency_monetary.csv'
    df.to_csv(file_name, index=False, header=False)

In [6]:
for i in range(263):
    rfm((_now - datetime.timedelta(days=i)).strftime("%Y-%m-%d"))

2021-05-21 2021 05 21
2021-05-20 2021 05 20
2021-05-19 2021 05 19
2021-05-18 2021 05 18
2021-05-17 2021 05 17
2021-05-16 2021 05 16
2021-05-15 2021 05 15
2021-05-14 2021 05 14
2021-05-13 2021 05 13
2021-05-12 2021 05 12
2021-05-11 2021 05 11
2021-05-10 2021 05 10
2021-05-09 2021 05 09
2021-05-08 2021 05 08
2021-05-07 2021 05 07
2021-05-06 2021 05 06
2021-05-05 2021 05 05
2021-05-04 2021 05 04
2021-05-03 2021 05 03
2021-05-02 2021 05 02
2021-05-01 2021 05 01
2021-04-30 2021 04 30
2021-04-29 2021 04 29
2021-04-28 2021 04 28
2021-04-27 2021 04 27
2021-04-26 2021 04 26
2021-04-25 2021 04 25
2021-04-24 2021 04 24
2021-04-23 2021 04 23
2021-04-22 2021 04 22
2021-04-21 2021 04 21
2021-04-20 2021 04 20
2021-04-19 2021 04 19
2021-04-18 2021 04 18
2021-04-17 2021 04 17
2021-04-16 2021 04 16
2021-04-15 2021 04 15
2021-04-14 2021 04 14
2021-04-13 2021 04 13
2021-04-12 2021 04 12
2021-04-11 2021 04 11
2021-04-10 2021 04 10
2021-04-09 2021 04 09
2021-04-08 2021 04 08
2021-04-07 2021 04 07
2021-04-06

In [8]:
connection.close()

# RFM
1. 3개월 단위로 데이터 처리
2. R : 가장 최근을 1.0으로 지정하고 하루를 지날수록 1.0 - i/180으로 값 지정
3. F : 3개월 주문수 합산.  하루 2번 이상 주문한 경우에도 그대로 합산. 최대 금액(X) 기준으로 N/X로 값 지정
4. M : 3개월 결제 금액 합산. 최대 금액(Y) 기준으로 N/Y로 값 지정
5. (R + F + M) / 3
6. 0.2 단위로 5단계 구분
7. 12월 ~ 5월까지 고객별 등급 저장

In [2]:
def recency(base, gap, rank_by_mber):
    if(rank_by_mber == 1):
        return 1.0 - gap/base
    else:
        return 0

In [3]:
unit = 180
stand_date = pd.to_datetime("2021-05-21")
files = glob.glob("/Users/choisj2/data/ml-mart/rds/rfm/**/**/**/*.csv")
colnames=['ORDER_REGIST_DATE', 'ORDRR_MBER_NO', 'ORDER_CNT', 'ORDER_DETAIL_CNT', 'ORDER_DETAIL_DSCNT_PC_SUM', 'ORDER_DETAIL_SELL_PC_SUM']
dtypes={'ORDER_REGIST_DATE':'datetime64', 'ORDRR_MBER_NO':'int64', 'ORDER_CNT':'int64', 'ORDER_DETAIL_CNT':'int64', 'ORDER_DETAIL_DSCNT_PC_SUM':'int64', 'ORDER_DETAIL_SELL_PC_SUM':'int64'}
bins = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
bins_label = ['1등급','2등급','3등급','4등급','5등급']
li = []

In [4]:
for file in files:
    df = pd.read_csv(file, names=colnames, header=None)
    li.append(df)

In [5]:
df = pd.concat(li, axis=0, ignore_index=True)

In [6]:
for col, col_type in dtypes.items():
    df[col] = df[col].astype(col_type)

In [7]:
# df['ORDER_REGIST_DATE'] = df['ORDER_REGIST_DATE'].astype('datetime64')
# df['ORDRR_MBER_NO'] = df['ORDRR_MBER_NO'].astype('int64')
# df['ORDER_CNT'] = df['ORDER_CNT'].astype('int64')
# df['ORDER_DETAIL_CNT'] = df['ORDER_DETAIL_CNT'].astype('int64')
# df['ORDER_DETAIL_DSCNT_PC_SUM'] = df['ORDER_DETAIL_DSCNT_PC_SUM'].astype('int64')
# df['ORDER_DETAIL_SELL_PC_SUM'] = df['ORDER_DETAIL_SELL_PC_SUM'].astype('int64')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6585731 entries, 0 to 6585730
Data columns (total 6 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   ORDER_REGIST_DATE          datetime64[ns]
 1   ORDRR_MBER_NO              int64         
 2   ORDER_CNT                  int64         
 3   ORDER_DETAIL_CNT           int64         
 4   ORDER_DETAIL_DSCNT_PC_SUM  int64         
 5   ORDER_DETAIL_SELL_PC_SUM   int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 301.5 MB


In [9]:
%%time
#df['ORDER_REGIST_DATE'] = pd.to_datetime(df['ORDER_REGIST_DATE'], infer_datetime_format=True)
df['GAP'] = (stand_date - df['ORDER_REGIST_DATE']).dt.days
df = df[(df['GAP'] <= unit) & (df['ORDRR_MBER_NO'] > 0)]
df = df.sort_values('ORDER_REGIST_DATE', ascending=False)
df['RANK_BY_MBER'] = df.groupby(['ORDRR_MBER_NO']).cumcount() + 1
df['RECENCY'] = df[['GAP', 'RANK_BY_MBER']].apply(lambda x : recency(unit, x[0], x[1]), axis=1)
df = df.groupby(['ORDRR_MBER_NO']).sum().reset_index()
df['FREQUENCY'] = df['ORDER_CNT'] / df['ORDER_CNT'].max()
df['MONETARY'] = df['ORDER_DETAIL_DSCNT_PC_SUM'] / df['ORDER_DETAIL_DSCNT_PC_SUM'].max()
df['RFM'] = (df['RECENCY'] + df['FREQUENCY'] + df['MONETARY']) / 3

CPU times: user 30.5 s, sys: 1.3 s, total: 31.8 s
Wall time: 31.8 s


In [10]:
df.head(5)

,ORDRR_MBER_NO,ORDER_CNT,ORDER_DETAIL_CNT,ORDER_DETAIL_DSCNT_PC_SUM,ORDER_DETAIL_SELL_PC_SUM,GAP,RANK_BY_MBER,RECENCY,FREQUENCY,MONETARY,RFM
0,1,13,27,2736530,2987620,1131,78,0.922222,0.002938,0.028568,0.317909
1,35,21,55,1344670,1506290,1916,190,0.977778,0.004746,0.014038,0.332187
2,80,11,12,507500,636300,787,36,0.883333,0.002486,0.005298,0.297039
3,88,24,33,2059200,2630700,1574,190,0.972222,0.005424,0.021497,0.333048
4,124,1,1,115100,132250,146,1,0.188889,0.000226,0.001202,0.063439


In [11]:
#df['GRADE'] = pd.cut(df['RFM'], bins, right=True, labels=bins_label)

In [12]:
df.head(5)

,ORDRR_MBER_NO,ORDER_CNT,ORDER_DETAIL_CNT,ORDER_DETAIL_DSCNT_PC_SUM,ORDER_DETAIL_SELL_PC_SUM,GAP,RANK_BY_MBER,RECENCY,FREQUENCY,MONETARY,RFM
0,1,13,27,2736530,2987620,1131,78,0.922222,0.002938,0.028568,0.317909
1,35,21,55,1344670,1506290,1916,190,0.977778,0.004746,0.014038,0.332187
2,80,11,12,507500,636300,787,36,0.883333,0.002486,0.005298,0.297039
3,88,24,33,2059200,2630700,1574,190,0.972222,0.005424,0.021497,0.333048
4,124,1,1,115100,132250,146,1,0.188889,0.000226,0.001202,0.063439
